In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import col

SPARK_MASTER = "spark://spark-master:7077" 

spark = (
    SparkSession.builder.appName("PySparkML_Test")
    .master(SPARK_MASTER)
    .getOrCreate()
)
print(f"--- Spark Session démarrée et connectée au Master : {SPARK_MASTER} ---")
print("-" * 50)

data = [
    (1.0, 5.0, 1.0),
    (2.0, 6.0, 0.0),
    (3.0, 7.0, 1.0),
    (4.0, 8.0, 0.0),
    (5.0, 9.0, 1.0),
    (6.0, 4.0, 0.0),
    (7.0, 3.0, 1.0),
    (8.0, 2.0, 0.0),
]
columns = ["feat_1", "feat_2", "label"]
df = spark.createDataFrame(data, columns)
print("Jeu de données initial:")
df.show()

assembler = VectorAssembler(inputCols=["feat_1", "feat_2"], outputCol="features")
output = assembler.transform(df)

training_data = output.select(
    col("label").cast("int"),
    col("features")
)
print("Données prêtes pour le ML:")
training_data.show(truncate=False)

lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)
print("Démarrage de l'entraînement du modèle...")

lr_model = lr.fit(training_data)

print(f"Coefficient pour les features : {lr_model.coefficients}")
print(f"Intercept : {lr_model.intercept}")

predictions = lr_model.transform(training_data)
print("\nPrédictions:")
predictions.select("label", "prediction", "probability", "features").show(truncate=False)

spark.stop()
print("-" * 50)
print("Spark Session arrêtée.")